Temp Notebook to Prep data files

- time_minutes_matrix.npz – N×N numpy array of travel times (minutes) between centers.

- distance_km_matrix.npz – N×N numpy array of distances (km) (optional but useful for costs).

- location_index.csv – mapping of center codes to matrix indices, columns: center,index.

- driver_states.json – dict keyed by driver/route ID → simple availability (or leave minimal and we default to 13h windows).

- cost_config.json (optional) – override cost weights if you want (deadhead, delay, outsourcing, etc.).

- trips.csv (optional for now) – your df_trips if you plan to generate candidates server-side later. For the current demo, the UI still sends disrupted_trips and candidates_per_trip in the request.


In [52]:
import numpy as np
import pandas as pd

In [53]:
#load csv file from data/private/df_rsl_clean.csv
df_rsl = pd.read_csv('../data/df_rsl_clean.csv')

In [54]:
#replace all blank fields with "no_data"
df_rsl = df_rsl.fillna("no_data")

In [55]:
df_rsl.head(20)

,Operator,Duty ID,Commencement Time,Ending Time,Element Type,Element Time,PLANZ Code,Due To Convey,Vehicle Type,Service Type,...,Driver Grade,Leg Mileage,From Postcode,From Lat,From Long,Mapped Name A,To Postcode,To Lat,To Long,Mapped Name B
0,ABVOC,AB201,05:06:00,05:36:00,START FACILITY,00:30:00,no_data,no_data,no_data,no_data,...,4: C+E,0,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre
1,ABVOC,AB201,05:36:00,06:06:00,LOAD(ASSIST),00:30:00,no_data,no_data,no_data,no_data,...,4: C+E,0,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre
2,ABVOC,AB201,06:06:00,09:30:00,TRAVEL,03:24:00,RN.AB.SDC.1,CONTAINER REPATRIATION,95 Artic DD,RM National Network,...,4: C+E,148,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre,ML2 0XX,55.773369,-3.950501,Scottish Hub
3,ABVOC,AB201,09:30:00,09:45:00,XCHANGE TRAILER,00:15:00,no_data,no_data,no_data,no_data,...,4: C+E,0,ML2 0XX,55.773369,-3.950501,Scottish Hub,ML2 0XX,55.773369,-3.950501,Scottish Hub
4,ABVOC,AB201,09:45:00,10:30:00,MEAL RELIEF WHILST VEHICLE UN/LOADED,00:45:00,no_data,no_data,no_data,no_data,...,4: C+E,0,ML2 0XX,55.773369,-3.950501,Scottish Hub,ML2 0XX,55.773369,-3.950501,Scottish Hub
5,ABVOC,AB201,10:30:00,10:45:00,LOAD(ASSIST),00:15:00,no_data,no_data,no_data,no_data,...,4: C+E,0,ML2 0XX,55.773369,-3.950501,Scottish Hub,ML2 0XX,55.773369,-3.950501,Scottish Hub
6,ABVOC,AB201,10:45:00,14:10:00,TRAVEL,03:25:00,SDC.AB.1,2C 48 MAIL,95 Artic DD,RM National Network,...,4: C+E,148,ML2 0XX,55.773369,-3.950501,Scottish Hub,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre
7,ABVOC,AB201,14:10:00,14:40:00,UNLOAD(ASSIST),00:30:00,no_data,no_data,no_data,no_data,...,4: C+E,0,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre
8,ABVOC,AB201,14:40:00,14:55:00,END FACILITY,00:15:00,no_data,no_data,no_data,no_data,...,4: C+E,0,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre
9,ABVOC,AB601,20:01:00,20:31:00,START FACILITY,00:30:00,no_data,no_data,no_data,no_data,...,4: C+E,0,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre,AB12 3TT,57.11134,-2.094009,Aberdeen Mail Centre


In [56]:
# list of column headers in the csv file
columns = df_rsl.columns.tolist()
print(columns)

['Operator', 'Duty ID', 'Commencement Time', 'Ending Time', 'Element Type', 'Element Time', 'PLANZ Code', 'Due To Convey', 'Vehicle Type', 'Service Type', 'From Site', 'To Site', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'From', 'To', 'Driver Grade', 'Leg Mileage', 'From Postcode', 'From Lat', 'From Long', 'Mapped Name A', 'To Postcode', 'To Lat', 'To Long', 'Mapped Name B']


# RSL EXPLAINER #

Each row is one single element of a driver's workday, with start and end times and days of operation. If a duty runs exactly the same on more than 1 day per week, it will have Y on the day it runs in the relevant columns labelled Mon, Tue, Wed etc. Please note that where duties cross over midnight, the individual row is labelled as Y or N based on whether that row element starts before midnight. So a Monday-only duty starting at 21:00 will show Y in the Mon column for all elements starting up to and including 23:59 and Y for all elements starting after 00:00.

All duties start with a 25 minute block of "Start Facility" time, and end with a 15 minute "End Facility" time.

List of key fields:

1. Operator - denotes the driver's home office location
2. Duty ID - denotes the unique ID for the entire route
3. Commencement Time - denotes the starting time for the task in HH:MM
4. Ending Time - denotes the ending time for the task in HH:MM
5. Element Type - describes the task. Could be Start Facility, End Facility, Meal Relief (break), Load (Assist), Travel, As Directed (available unassigned time) or other non-driving tasks
6. Element Time - the duration of the task in HH:MM
7. PLANZ Code - this is an internal field that describes load type, but is not useful for our model
8. Due to Convey - describes the load type for Travel legs. May sometimes be blank. This is how we determine priority.
9. Vehicle Type - size of vehicle
10. Service Type - ignore
11. From Site - Dispatch location
12. To Site - Arrival location
13 to 19 inclusive - Days of Operation - contains Y if the element operates on this day, N if it does not.
Additional Columns - contain postcodes and Lat/Long co-ordinates where available, and a "Mapped Name" for From and To Locations. This is because the RSL uses a different naming convention to our other database (!!)

## Vehicle Pool Validation (time + geography + empty/loaded)
This section validates helper-vehicle eligibility for a target run by:
- Classifying travel legs as `EMPTY` vs `LOADED` (using **Due to Convey** first, with safe fallbacks)
- Applying day-of-operation and cross-midnight carry rules
- Filtering by a requested time window
- Filtering by proximity to the pickup area

Use this to answer: “which vehicles can genuinely help this run?”

In [57]:
from datetime import datetime
import math


def _canon(text):
    return "".join(ch for ch in str(text or "").lower() if ch.isalnum())


def pick_col(df, *candidates):
    by_canon = {_canon(c): c for c in df.columns}
    for cand in candidates:
        found = by_canon.get(_canon(cand))
        if found:
            return found
    return None


def parse_hhmmss_to_min(v):
    s = str(v or "").strip()
    if not s:
        return math.nan
    parts = s.split(":")
    if len(parts) < 2:
        return math.nan
    try:
        hh = int(parts[0])
        mm = int(parts[1])
        return hh * 60 + mm
    except Exception:
        return math.nan


def truthy(v):
    if isinstance(v, (int, float)):
        try:
            return int(v) != 0
        except Exception:
            return False
    return str(v).strip().lower() in {"y", "yes", "true", "1", "t"}


def haversine_miles(lat1, lon1, lat2, lon2):
    if any(pd.isna(x) for x in [lat1, lon1, lat2, lon2]):
        return math.nan
    r = 3958.8
    p1 = math.radians(lat1)
    p2 = math.radians(lat2)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(p1) * math.cos(p2) * math.sin(dlon / 2) ** 2
    return 2 * r * math.asin(math.sqrt(a))


def classify_load_state(row, due_col=None, planz_col=None):
    due = str(row.get(due_col, "") if due_col else "").upper().strip()
    planz = str(row.get(planz_col, "") if planz_col else "").upper().strip()
    et = str(row.get("Element Type", "")).upper().strip()

    if "TRAVEL" not in et:
        return "NON_TRAVEL"

    if any(k in due for k in ["EMPTY", "DEADHEAD", "RETURN"]):
        return "EMPTY"
    if due not in {"", "NO_DATA", "NAN", "NONE", "NULL"}:
        return "LOADED"

    if any(k in planz for k in ["EMPTY", "DEADHEAD", "RETURN"]):
        return "EMPTY"

    return "UNKNOWN_TRAVEL"


def weekday_tag(dt_str):
    d = datetime.fromisoformat(dt_str)
    return ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"][d.weekday()]


def previous_day(day):
    days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    i = days.index(day)
    return days[(i - 1) % 7]


# --- Scenario controls ---
REQUEST_WHEN_LOCAL = "2025-09-02T10:30"  # Tuesday
REQUEST_FROM = "BIRMINGHAM MAIL CENTRE"
REQUEST_TO = "MIDLANDS SUPER HUB"
TIME_WINDOW_MIN = 120
MAX_PICKUP_RADIUS_MI = 80
EMPTY_ONLY = True
POST_MIDNIGHT_CARRY_CUTOFF_MIN = 6 * 60


# --- Column mapping ---
col_duty = pick_col(df_rsl, "Duty ID")
col_operator = pick_col(df_rsl, "Operator")
col_vehicle = pick_col(df_rsl, "Vehicle Type")
col_et = pick_col(df_rsl, "Element Type")
col_comm = pick_col(df_rsl, "Commencement Time")
col_end = pick_col(df_rsl, "Ending Time")
col_from = pick_col(df_rsl, "Mapped Name A", "From Site")
col_to = pick_col(df_rsl, "Mapped Name B", "To Site")
col_due = pick_col(df_rsl, "Due To Convey", "Due to Convey")
col_planz = pick_col(df_rsl, "PLANZ Code", "Planz Code")
col_lat_a = pick_col(df_rsl, "Lat_A", "From Lat", "Latitude A")
col_lon_a = pick_col(df_rsl, "Long_A", "From Long", "Lon_A", "Longitude A")
col_lat_b = pick_col(df_rsl, "Lat_B", "To Lat", "Latitude B")
col_lon_b = pick_col(df_rsl, "Long_B", "To Long", "Lon_B", "Longitude B")

required = [col_duty, col_et, col_comm, col_end, col_from, col_to]
if any(c is None for c in required):
    raise ValueError(f"Missing required columns. Found mappings: duty={col_duty}, et={col_et}, comm={col_comm}, end={col_end}, from={col_from}, to={col_to}")


# --- Working frame ---
work = df_rsl.copy()
work["start_min"] = work[col_comm].apply(parse_hhmmss_to_min)
work["end_min"] = work[col_end].apply(parse_hhmmss_to_min)
work["element_type_norm"] = work[col_et].astype(str).str.upper().str.strip()
work["from_norm"] = work[col_from].astype(str).str.upper().str.strip()
work["to_norm"] = work[col_to].astype(str).str.upper().str.strip()
work["load_state"] = work.apply(lambda r: classify_load_state(r, due_col=col_due, planz_col=col_planz), axis=1)

if col_lat_a and col_lon_a:
    work["from_lat"] = pd.to_numeric(work[col_lat_a], errors="coerce")
    work["from_lon"] = pd.to_numeric(work[col_lon_a], errors="coerce")
else:
    work["from_lat"] = math.nan
    work["from_lon"] = math.nan

if col_lat_b and col_lon_b:
    work["to_lat"] = pd.to_numeric(work[col_lat_b], errors="coerce")
    work["to_lon"] = pd.to_numeric(work[col_lon_b], errors="coerce")
else:
    work["to_lat"] = math.nan
    work["to_lon"] = math.nan

req_day = weekday_tag(REQUEST_WHEN_LOCAL)
prev_day = previous_day(req_day)
req_min = parse_hhmmss_to_min(datetime.fromisoformat(REQUEST_WHEN_LOCAL).strftime("%H:%M:00"))

if req_day not in work.columns:
    raise ValueError(f"Day column '{req_day}' not found in RSL columns")

work["active_on_req_day"] = work[req_day].apply(truthy)
work["post_midnight_carry"] = (
    work["start_min"].between(0, POST_MIDNIGHT_CARRY_CUTOFF_MIN, inclusive="left")
    & work[prev_day].apply(truthy)
)
work["active_effective"] = work["active_on_req_day"] | work["post_midnight_carry"]

# Locate requested origin coordinates from any matching from/to row
origin_rows = work[(work["from_norm"] == REQUEST_FROM) | (work["to_norm"] == REQUEST_FROM)]
origin_lat = origin_rows["from_lat"].dropna().iloc[0] if not origin_rows["from_lat"].dropna().empty else (
    origin_rows["to_lat"].dropna().iloc[0] if not origin_rows["to_lat"].dropna().empty else math.nan
)
origin_lon = origin_rows["from_lon"].dropna().iloc[0] if not origin_rows["from_lon"].dropna().empty else (
    origin_rows["to_lon"].dropna().iloc[0] if not origin_rows["to_lon"].dropna().empty else math.nan
)

travel = work[work["element_type_norm"].str.contains("TRAVEL", na=False)].copy()
travel = travel[travel["active_effective"]]
travel["time_gap_min"] = (travel["start_min"] - req_min).abs()
travel = travel[travel["time_gap_min"] <= TIME_WINDOW_MIN]

if EMPTY_ONLY:
    travel = travel[travel["load_state"] == "EMPTY"]

travel["distance_to_pickup_mi"] = travel.apply(
    lambda r: haversine_miles(r["from_lat"], r["from_lon"], origin_lat, origin_lon), axis=1
)
travel = travel[travel["distance_to_pickup_mi"].isna() | (travel["distance_to_pickup_mi"] <= MAX_PICKUP_RADIUS_MI)]

helper_cols = [
    col_duty,
    col_operator,
    col_vehicle,
    col_comm,
    col_end,
    col_from,
    col_to,
    "load_state",
    "distance_to_pickup_mi",
    "time_gap_min",
    "active_on_req_day",
    "post_midnight_carry",
]
helper_cols = [c for c in helper_cols if c in travel.columns]

helpers = travel[helper_cols].copy().sort_values(["distance_to_pickup_mi", "time_gap_min"], na_position="last")

print(f"Scenario: {REQUEST_FROM} -> {REQUEST_TO} at {REQUEST_WHEN_LOCAL} ({req_day})")
print(f"Travel rows active on day (effective): {len(work[work['active_effective'] & work['element_type_norm'].str.contains('TRAVEL', na=False)])}")
print(f"Potential helper legs after filters: {len(helpers)}")

helpers.head(50)

Scenario: BIRMINGHAM MAIL CENTRE -> MIDLANDS SUPER HUB at 2025-09-02T10:30 (Tue)
Travel rows active on day (effective): 11039
Potential helper legs after filters: 17


,Duty ID,Operator,Vehicle Type,Commencement Time,Ending Time,Mapped Name A,Mapped Name B,load_state,distance_to_pickup_mi,time_gap_min,active_on_req_day,post_midnight_carry
35025,MSHSA1005,MSH,95 Artic DD,10:49:00,10:54:00,Midlands Super Hub,National Distribution Centre,EMPTY,0.000000,19,True,False
68227,YDC034,YDC,95 Artic DD,09:45:00,12:00:00,Midlands Super Hub,Yorkshire Hub,EMPTY,0.000000,45,True,False
35021,MSHSA1005,MSH,95 Artic DD,09:31:00,09:36:00,Midlands Super Hub,National Distribution Centre,EMPTY,0.000000,59,True,False
68445,YDC047,YDC,95 Artic DD,08:58:00,11:13:00,Midlands Super Hub,Yorkshire Hub,EMPTY,0.000000,92,True,False
35029,MSHSA1005,MSH,95 Artic DD,12:07:00,12:12:00,Midlands Super Hub,National Distribution Centre,EMPTY,0.000000,97,True,False
27649,MSH143,MSH,24 17t Rigid,10:05:00,11:00:00,National Distribution Centre,Roman Originals (B24 9QP),EMPTY,0.508903,25,True,False
25904,MSH011B,MSH,49 Artic,10:00:00,10:50:00,National Distribution Centre,no_data,EMPTY,0.508903,30,True,False
28068,MSH155,MSH,49 Artic,11:25:00,12:15:00,National Distribution Centre,no_data,EMPTY,0.508903,55,True,False
27421,MSH135,MSH,49 Artic,09:30:00,10:20:00,National Distribution Centre,Sky Broadband CV21 1DZ,EMPTY,0.508903,60,True,False
27327,MSH129,MSH,95 Artic DD,09:15:00,10:45:00,National Distribution Centre,Supersmart (WS9 8DT),EMPTY,0.508903,75,True,False


In [58]:
# Optional second-pass filter: exclude helpers whose HOME BASE is too far from pickup area
MAX_HOME_BASE_RADIUS_MI = 70

starts = work[(work["element_type_norm"].str.contains("START FACILITY", na=False)) & (work["active_effective"])].copy()
starts = starts.sort_values([col_duty, "start_min"]).drop_duplicates(subset=[col_duty], keep="first")

home_cols = [col_duty, col_operator, col_vehicle, "from_norm", "from_lat", "from_lon"]
home_cols = [c for c in home_cols if c in starts.columns]
home_base = starts[home_cols].rename(columns={
    "from_norm": "home_base_name",
    "from_lat": "home_base_lat",
    "from_lon": "home_base_lon",
})

helpers_with_home = helpers.merge(home_base, on=col_duty, how="left", suffixes=("", "_home"))
helpers_with_home["home_base_to_pickup_mi"] = helpers_with_home.apply(
    lambda r: haversine_miles(r.get("home_base_lat"), r.get("home_base_lon"), origin_lat, origin_lon), axis=1
)

helpers_strict = helpers_with_home[
    helpers_with_home["home_base_to_pickup_mi"].isna()
    | (helpers_with_home["home_base_to_pickup_mi"] <= MAX_HOME_BASE_RADIUS_MI)
].copy()

print(f"Helpers before home-base filter: {len(helpers)}")
print(f"Helpers after home-base filter ({MAX_HOME_BASE_RADIUS_MI} mi): {len(helpers_strict)}")

view_cols = [
    col_duty,
    col_operator,
    col_vehicle,
    "home_base_name",
    "home_base_to_pickup_mi",
    col_from,
    col_to,
    "load_state",
    "distance_to_pickup_mi",
    "time_gap_min",
]
view_cols = [c for c in view_cols if c in helpers_strict.columns]

helpers_strict.sort_values(["home_base_to_pickup_mi", "distance_to_pickup_mi", "time_gap_min"], na_position="last")[view_cols].head(50)

Helpers before home-base filter: 17
Helpers after home-base filter (70 mi): 14


,Duty ID,Operator,Vehicle Type,home_base_name,home_base_to_pickup_mi,Mapped Name A,Mapped Name B,load_state,distance_to_pickup_mi,time_gap_min
0,MSHSA1005,MSH,95 Artic DD,MIDLANDS SUPER HUB,0.000000,Midlands Super Hub,National Distribution Centre,EMPTY,0.000000,19
2,MSHSA1005,MSH,95 Artic DD,MIDLANDS SUPER HUB,0.000000,Midlands Super Hub,National Distribution Centre,EMPTY,0.000000,59
4,MSHSA1005,MSH,95 Artic DD,MIDLANDS SUPER HUB,0.000000,Midlands Super Hub,National Distribution Centre,EMPTY,0.000000,97
5,MSH143,MSH,24 17t Rigid,MIDLANDS SUPER HUB,0.000000,National Distribution Centre,Roman Originals (B24 9QP),EMPTY,0.508903,25
6,MSH011B,MSH,49 Artic,MIDLANDS SUPER HUB,0.000000,National Distribution Centre,no_data,EMPTY,0.508903,30
7,MSH155,MSH,49 Artic,MIDLANDS SUPER HUB,0.000000,National Distribution Centre,no_data,EMPTY,0.508903,55
8,MSH135,MSH,49 Artic,MIDLANDS SUPER HUB,0.000000,National Distribution Centre,Sky Broadband CV21 1DZ,EMPTY,0.508903,60
9,MSH129,MSH,95 Artic DD,MIDLANDS SUPER HUB,0.000000,National Distribution Centre,Supersmart (WS9 8DT),EMPTY,0.508903,75
10,MSH126A,MSH,49 Artic,MIDLANDS SUPER HUB,0.000000,National Distribution Centre,Advanced Direct DY2 9RE,EMPTY,0.508903,110
14,MSH015A,MSH,95 Artic DD,MIDLANDS SUPER HUB,0.000000,Greenford Mail Centre,National Distribution Centre,EMPTY,66.998923,83


## Feasibility Stage: Pickup-first, then cascade potential
This stage intentionally uses the **collection point** (`REQUEST_FROM`) as the geographic anchor.

Process:
1. Find duties with travel legs near Birmingham pickup within the time window.
2. For those duties, inspect the rest of the schedule on the same effective day.
3. Summarize cascade potential (lower-priority travel that could be displaced, plus As Directed minutes).

This gives a short-list of drivers who are both geographically/time feasible **and** operationally flexible.

In [59]:
# 1) Pickup-first duty pool: near REQUEST_FROM at requested time window
CANDIDATE_PICKUP_RADIUS_MI = 40

# Defensive recompute so this cell is runnable independently
if "time_gap_min" not in work.columns:
    work["time_gap_min"] = (work["start_min"] - req_min).abs()

if "distance_to_pickup_mi" not in work.columns:
    work["distance_to_pickup_mi"] = work.apply(
        lambda r: haversine_miles(r.get("from_lat"), r.get("from_lon"), origin_lat, origin_lon), axis=1
    )

# Priority fallback from available RSL fields if explicit numeric priority is absent
if "priority" not in work.columns:
    def _priority_proxy(row):
        due = str(row.get(col_due, "") if col_due else "").upper().strip()
        if any(k in due for k in ["SPECIAL", "PRIORITY", "P1", "P2"]):
            return 2
        if any(k in due for k in ["EMPTY", "DEADHEAD", "RETURN"]):
            return 5
        if due in {"", "NO_DATA", "NAN", "NONE", "NULL"}:
            return 3
        return 3
    work["priority_proxy"] = work.apply(_priority_proxy, axis=1)
else:
    work["priority_proxy"] = pd.to_numeric(work["priority"], errors="coerce").fillna(3)

pickup_candidates = work[
    (work["element_type_norm"].str.contains("TRAVEL", na=False))
    & (work["active_effective"])
    & (work["time_gap_min"] <= TIME_WINDOW_MIN)
    & (
        work["distance_to_pickup_mi"].isna()
        | (work["distance_to_pickup_mi"] <= CANDIDATE_PICKUP_RADIUS_MI)
    )
].copy()

# Optional: keep only EMPTY/UNKNOWN travel for the immediate helper leg
pickup_candidates = pickup_candidates[pickup_candidates["load_state"].isin(["EMPTY", "UNKNOWN_TRAVEL"])]

# 2) Build duty-level feasibility summary (what else can be cascaded?)
summary_rows = []
for duty_id, duty_rows in pickup_candidates.groupby(col_duty):
    duty_all = work[(work[col_duty] == duty_id) & (work["active_effective"])].copy()

    # Pickup-side evidence
    nearest_pickup = duty_rows["distance_to_pickup_mi"].min(skipna=True)
    best_time_gap = duty_rows["time_gap_min"].min(skipna=True)
    pickup_leg_count = len(duty_rows)

    # What can be displaced/cascaded after request time?
    future = duty_all[duty_all["start_min"] >= req_min]
    future_travel = future[future["element_type_norm"].str.contains("TRAVEL", na=False)]

    lower_priority_travel = future_travel[future_travel["priority_proxy"].astype(float) >= 4]
    empty_future_travel = future_travel[future_travel["load_state"] == "EMPTY"]

    as_directed_future = future[future["element_type_norm"].str.contains("AS DIRECTED", na=False)].copy()
    as_directed_future["dur_min"] = (
        as_directed_future["end_min"].fillna(as_directed_future["start_min"])
        - as_directed_future["start_min"].fillna(0)
    )
    as_directed_minutes = as_directed_future["dur_min"].clip(lower=0).sum()

    # Home-base check (reuse from prior cell if available)
    home_base_name = None
    home_base_to_pickup = math.nan
    if "home_base" in globals() and not home_base.empty:
        hb = home_base[home_base[col_duty] == duty_id]
        if not hb.empty:
            home_base_name = hb.iloc[0].get("home_base_name")
            home_base_to_pickup = haversine_miles(
                hb.iloc[0].get("home_base_lat"),
                hb.iloc[0].get("home_base_lon"),
                origin_lat,
                origin_lon,
            )

    cascade_potential_score = (
        int(len(lower_priority_travel)) * 3
        + int(len(empty_future_travel)) * 2
        + (as_directed_minutes / 30.0)
    )

    summary_rows.append(
        {
            col_duty: duty_id,
            "operator": duty_rows[col_operator].iloc[0] if col_operator in duty_rows.columns else None,
            "vehicle_type": duty_rows[col_vehicle].iloc[0] if col_vehicle in duty_rows.columns else None,
            "nearest_pickup_mi": nearest_pickup,
            "best_time_gap_min": best_time_gap,
            "pickup_leg_count": pickup_leg_count,
            "future_travel_count": len(future_travel),
            "future_lower_priority_travel_count": len(lower_priority_travel),
            "future_empty_travel_count": len(empty_future_travel),
            "future_as_directed_minutes": float(as_directed_minutes),
            "home_base_name": home_base_name,
            "home_base_to_pickup_mi": home_base_to_pickup,
            "cascade_potential_score": float(cascade_potential_score),
        }
    )

feasibility = pd.DataFrame(summary_rows)

if feasibility.empty:
    print("No pickup-feasible duties found in this time window/radius.")
else:
    feasibility = feasibility.sort_values(
        ["nearest_pickup_mi", "best_time_gap_min", "cascade_potential_score"],
        ascending=[True, True, False],
        na_position="last",
    )

    print(f"Pickup-feasible duties (before further constraints): {len(feasibility)}")
    display_cols = [
        col_duty,
        "operator",
        "vehicle_type",
        "nearest_pickup_mi",
        "best_time_gap_min",
        "future_lower_priority_travel_count",
        "future_empty_travel_count",
        "future_as_directed_minutes",
        "home_base_name",
        "home_base_to_pickup_mi",
        "cascade_potential_score",
    ]
    display_cols = [c for c in display_cols if c in feasibility.columns]
    feasibility[display_cols].head(100)

Pickup-feasible duties (before further constraints): 10


## Due to Convey → Priority matrix audit
This cell loads the active `priority_map.json` used by the planner and compares it to observed `Due to Convey` values in the RSL, so we can see mapping coverage and gaps.

In [60]:
import json
from pathlib import Path

# Where planner loads from in this repo layout
priority_map_path = Path("../data/private/active/priority_map.json")
if not priority_map_path.exists():
    priority_map_path = Path("../data/private/priority_map.json")
if not priority_map_path.exists():
    priority_map_path = Path("../data/priority_map.json")

if not priority_map_path.exists():
    raise FileNotFoundError("No priority_map.json found in expected data paths")

priority_map_raw = json.loads(priority_map_path.read_text(encoding="utf-8"))
priority_map = {str(k).upper().strip(): int(v) for k, v in priority_map_raw.items()}

print(f"Loaded priority map from: {priority_map_path}")
print(f"Entries: {len(priority_map)}")

# Detect Due to Convey column robustly
due_col_audit = pick_col(df_rsl, "Due To Convey", "Due to Convey")
if due_col_audit is None:
    raise ValueError("Could not find Due to Convey column in df_rsl")

observed_due = (
    df_rsl[due_col_audit]
    .astype(str)
    .str.upper()
    .str.strip()
    .replace({"": "NO_DATA", "NAN": "NO_DATA", "NONE": "NO_DATA", "NULL": "NO_DATA"})
)

counts = observed_due.value_counts().rename_axis("due_to_convey").reset_index(name="rows")
counts["mapped_priority"] = counts["due_to_convey"].map(priority_map)
counts["mapped"] = counts["mapped_priority"].notna()
counts["effective_priority"] = counts["mapped_priority"].fillna(priority_map.get("DEFAULT", 3)).astype(int)

coverage = pd.DataFrame(
    {
        "total_distinct_due_values": [int(counts.shape[0])],
        "mapped_distinct_due_values": [int(counts["mapped"].sum())],
        "unmapped_distinct_due_values": [int((~counts["mapped"]).sum())],
        "mapped_rows": [int(counts.loc[counts["mapped"], "rows"].sum())],
        "unmapped_rows": [int(counts.loc[~counts["mapped"], "rows"].sum())],
        "default_priority_used_for_unmapped": [int(priority_map.get("DEFAULT", 3))],
    }
)

print("\nCoverage summary:")
display(coverage)

print("Top observed Due to Convey values with mapping:")
display(counts.sort_values("rows", ascending=False).head(40))

print("Unmapped Due to Convey values (needs matrix updates):")
display(counts[~counts["mapped"]].sort_values("rows", ascending=False))

Loaded priority map from: ..\data\private\active\priority_map.json
Entries: 25

Coverage summary:


,total_distinct_due_values,mapped_distinct_due_values,unmapped_distinct_due_values,mapped_rows,unmapped_rows,default_priority_used_for_unmapped
0,25,25,0,73900,0,3


Top observed Due to Convey values with mapping:


,due_to_convey,rows,mapped_priority,mapped,effective_priority
0,NO_DATA,50421,3,True,3
1,TRAVEL_NO_DATA,8635,3,True,3
2,1C 24 MAIL,2973,1,True,1
3,2C 48 MAIL,2718,2,True,2
4,CONTAINER REPATRIATION,2483,4,True,4
5,PF 24 PARCELS,1271,1,True,1
6,DELIVERY,1195,1,True,1
7,RDC 24 TRACKED,1063,1,True,1
8,COLLECTION,805,2,True,2
9,RDC 48 TRACKED,681,2,True,2


Unmapped Due to Convey values (needs matrix updates):


,due_to_convey,rows,mapped_priority,mapped,effective_priority


## Build and export a canonical priority map
This cell creates a clean, frequency-aware map from observed `Due to Convey` values.

Rules used:
- Keep existing explicit mappings where present
- Keep `NO_DATA` and `TRAVEL_NO_DATA` at priority `3`
- For any new unmapped values, assign a proposed default priority (`3`) for review
- Export sorted JSON for maintenance

In [61]:
from pathlib import Path
import json
import pandas as pd

# Reuse objects from earlier cells: counts, priority_map
if 'counts' not in globals() or 'priority_map' not in globals():
    raise RuntimeError("Run the prior Due to Convey audit cell first (needs `counts` and `priority_map`).")

DEFAULT_PROPOSED_PRIORITY = 3
SPECIAL_FIXED = {
    'NO_DATA': 3,
    'TRAVEL_NO_DATA': 3,
}

# Normalize map keys for canonical matching (case-insensitive input -> uppercase canonical keys)
priority_map_norm = {str(k).strip().upper(): int(v) for k, v in priority_map.items()}

# Normalize counts into a 2-column DataFrame: due_to_convey, observed_count
if isinstance(counts, pd.Series):
    canonical_df = (
        counts.rename('observed_count')
        .rename_axis('due_to_convey')
        .reset_index()
    )
elif isinstance(counts, pd.DataFrame):
    c = counts.copy()
    cols = list(c.columns)

    if {'due_to_convey', 'count'}.issubset(cols):
        canonical_df = c[['due_to_convey', 'count']].rename(columns={'count': 'observed_count'})
    elif {'due_to_convey', 'observed_count'}.issubset(cols):
        canonical_df = c[['due_to_convey', 'observed_count']]
    else:
        # Fallback: pick first text-like column as label and first numeric column as count
        text_cols = [col for col in cols if c[col].dtype == 'object' or str(c[col].dtype).startswith('string')]
        num_cols = [col for col in cols if pd.api.types.is_numeric_dtype(c[col])]

        if text_cols and num_cols:
            canonical_df = c[[text_cols[0], num_cols[0]]].rename(columns={text_cols[0]: 'due_to_convey', num_cols[0]: 'observed_count'})
        elif len(cols) >= 2:
            canonical_df = c[[cols[0], cols[1]]].rename(columns={cols[0]: 'due_to_convey', cols[1]: 'observed_count'})
        else:
            raise RuntimeError(f"Could not infer `counts` schema. Found columns: {cols}")
else:
    raise RuntimeError("`counts` must be a pandas Series or DataFrame.")

canonical_df['due_to_convey'] = canonical_df['due_to_convey'].astype(str).str.strip().str.upper()
canonical_df['observed_count'] = pd.to_numeric(canonical_df['observed_count'], errors='coerce').fillna(0).astype(int)

# Collapse duplicates after normalization
canonical_df = canonical_df.groupby('due_to_convey', as_index=False)['observed_count'].sum()

canonical_df['current_priority'] = canonical_df['due_to_convey'].map(priority_map_norm)
canonical_df['proposed_priority'] = canonical_df['due_to_convey'].map(priority_map_norm)

# Enforce special fixed values
for key, val in SPECIAL_FIXED.items():
    canonical_df.loc[canonical_df['due_to_convey'] == key, 'proposed_priority'] = val

# Fill unmapped with default for review
canonical_df['proposed_priority'] = canonical_df['proposed_priority'].fillna(DEFAULT_PROPOSED_PRIORITY).astype(int)

# Sort by frequency, then label
canonical_df = canonical_df.sort_values(['observed_count', 'due_to_convey'], ascending=[False, True]).reset_index(drop=True)

# Build deterministic JSON map (alphabetical keys)
proposed_map = {
    row['due_to_convey']: int(row['proposed_priority'])
    for _, row in canonical_df.sort_values('due_to_convey').iterrows()
}

out_dir = Path('..') / 'data' / 'private' / 'active'
out_dir.mkdir(parents=True, exist_ok=True)
proposed_path = out_dir / 'priority_map.proposed.json'
canonical_csv_path = out_dir / 'priority_map.canonical_table.csv'

with proposed_path.open('w', encoding='utf-8') as f:
    json.dump(proposed_map, f, indent=2, ensure_ascii=False)

canonical_df.to_csv(canonical_csv_path, index=False)

print(f"Proposed map written: {proposed_path.resolve()}")
print(f"Canonical table written: {canonical_csv_path.resolve()}")
print(f"Observed classes: {len(canonical_df)}")
print(f"Unmapped previously: {canonical_df['current_priority'].isna().sum()}")

canonical_df.head(20)

Proposed map written: C:\Users\ellag\Desktop\ML Course\demo_deploy\demo_deploy\data\private\active\priority_map.proposed.json
Canonical table written: C:\Users\ellag\Desktop\ML Course\demo_deploy\demo_deploy\data\private\active\priority_map.canonical_table.csv
Observed classes: 25
Unmapped previously: 0


,due_to_convey,observed_count,current_priority,proposed_priority
0,NO_DATA,50421,3,3
1,TRAVEL_NO_DATA,8635,3,3
2,1C 24 MAIL,2973,1,1
3,2C 48 MAIL,2718,2,2
4,CONTAINER REPATRIATION,2483,4,4
5,PF 24 PARCELS,1271,1,1
6,DELIVERY,1195,1,1
7,RDC 24 TRACKED,1063,1,1
8,COLLECTION,805,2,2
9,RDC 48 TRACKED,681,2,2


In [62]:
# Compare current active map with proposed map and show exact diffs
from pathlib import Path
import json
import pandas as pd

active_path = Path('..') / 'data' / 'private' / 'active' / 'priority_map.json'
proposed_path = Path('..') / 'data' / 'private' / 'active' / 'priority_map.proposed.json'

with active_path.open('r', encoding='utf-8') as f:
    active_map = json.load(f)

with proposed_path.open('r', encoding='utf-8') as f:
    proposed_map = json.load(f)

active_keys = set(active_map.keys())
proposed_keys = set(proposed_map.keys())

added = sorted(proposed_keys - active_keys)
removed = sorted(active_keys - proposed_keys)
common = sorted(active_keys & proposed_keys)
changed = sorted([k for k in common if int(active_map[k]) != int(proposed_map[k])])

print(f"Active keys: {len(active_keys)}")
print(f"Proposed keys: {len(proposed_keys)}")
print(f"Added keys: {len(added)}")
print(f"Removed keys: {len(removed)}")
print(f"Changed priorities: {len(changed)}")

diff_rows = []
for key in added:
    diff_rows.append({'due_to_convey': key, 'active_priority': None, 'proposed_priority': int(proposed_map[key]), 'change_type': 'ADDED'})
for key in removed:
    diff_rows.append({'due_to_convey': key, 'active_priority': int(active_map[key]), 'proposed_priority': None, 'change_type': 'REMOVED'})
for key in changed:
    diff_rows.append({'due_to_convey': key, 'active_priority': int(active_map[key]), 'proposed_priority': int(proposed_map[key]), 'change_type': 'CHANGED'})

diff_df = pd.DataFrame(diff_rows).sort_values(['change_type', 'due_to_convey']) if diff_rows else pd.DataFrame(columns=['due_to_convey', 'active_priority', 'proposed_priority', 'change_type'])

if diff_df.empty:
    print('No differences between active and proposed maps.')
else:
    display(diff_df)

# Optional review helper: uncomment if you want to promote proposed to active after review
# with active_path.open('w', encoding='utf-8') as f:
#     json.dump(proposed_map, f, indent=2, ensure_ascii=False)
# print(f'Promoted proposed map to active: {active_path.resolve()}')

Active keys: 25
Proposed keys: 25
Added keys: 1
Removed keys: 1
Changed priorities: 0


,due_to_convey,active_priority,proposed_priority,change_type
0,TRAVEL_NO_DATA,NaN,3.0,ADDED
1,travel_no_data,3.0,NaN,REMOVED


## First-pass filter v2 (priority + As Directed + delivery-side home base)
This cell applies the revised first-pass rules:
- Include travel legs that are either `EMPTY` **or** priority-eligible for displacement (`leg_priority >= request_priority`)
- Include `AS DIRECTED` blocks near pickup, but only if duration is at least 30 minutes
- Keep geography configurable (default 10 miles, range concept 0–80)
- Apply home-base filter against **delivery** location (not pickup), with configurable radius

In [65]:
import numpy as np

# --- Tunables (match UI defaults/intent) ---
REQUEST_PRIORITY = 3
PICKUP_RADIUS_MI_V2 = 10
HOME_BASE_TO_DELIVERY_RADIUS_MI_V2 = 30
MIN_AS_DIRECTED_MIN = 30

if 'work' not in globals():
    raise RuntimeError("Run Cell 9 first to build `work`, location mappings, and helper functions.")

# Ensure distances/time-gap exist
if 'time_gap_min' not in work.columns:
    work['time_gap_min'] = (work['start_min'] - req_min).abs()
if 'distance_to_pickup_mi' not in work.columns:
    work['distance_to_pickup_mi'] = work.apply(
        lambda r: haversine_miles(r.get('from_lat'), r.get('from_lon'), origin_lat, origin_lon), axis=1
    )

# Delivery anchor coords (for home-base filter)
delivery_rows = work[(work['from_norm'] == REQUEST_TO) | (work['to_norm'] == REQUEST_TO)]
delivery_lat = delivery_rows['from_lat'].dropna().iloc[0] if not delivery_rows['from_lat'].dropna().empty else (
    delivery_rows['to_lat'].dropna().iloc[0] if not delivery_rows['to_lat'].dropna().empty else math.nan
)
delivery_lon = delivery_rows['from_lon'].dropna().iloc[0] if not delivery_rows['from_lon'].dropna().empty else (
    delivery_rows['to_lon'].dropna().iloc[0] if not delivery_rows['to_lon'].dropna().empty else math.nan
)

# Normalize travel priority if absent
if 'priority' in work.columns:
    work['priority_num'] = pd.to_numeric(work['priority'], errors='coerce').fillna(3).astype(int)
else:
    if 'priority_proxy' not in work.columns:
        def _priority_proxy_v2(row):
            due = str(row.get(col_due, '') if col_due else '').upper().strip()
            if any(k in due for k in ['P1', 'P2', 'SPECIAL', 'PRIORITY']):
                return 2
            if any(k in due for k in ['EMPTY', 'DEADHEAD', 'RETURN']):
                return 5
            return 3
        work['priority_proxy'] = work.apply(_priority_proxy_v2, axis=1)
    work['priority_num'] = pd.to_numeric(work['priority_proxy'], errors='coerce').fillna(3).astype(int)

# AS DIRECTED duration
as_directed_mask = work['element_type_norm'].str.contains('AS DIRECTED', na=False)
work['as_directed_duration_min'] = np.where(
    as_directed_mask,
    (work['end_min'].fillna(work['start_min']) - work['start_min'].fillna(0)).clip(lower=0),
    0,
)

# Waterfall counts
w0 = work.copy()
c0 = len(w0)

w1 = w0[w0['active_effective']].copy()
c1 = len(w1)

# Keep travel or AS DIRECTED
travel_or_asd = w1[
    w1['element_type_norm'].str.contains('TRAVEL', na=False)
    | w1['element_type_norm'].str.contains('AS DIRECTED', na=False)
].copy()
c2 = len(travel_or_asd)

# Time window
time_ok = travel_or_asd['time_gap_min'] <= TIME_WINDOW_MIN
w3 = travel_or_asd[time_ok].copy()
c3 = len(w3)

# Priority/empty/as-directed eligibility (1 highest urgency, 5 lowest)
is_travel = w3['element_type_norm'].str.contains('TRAVEL', na=False)
is_asd = w3['element_type_norm'].str.contains('AS DIRECTED', na=False)

travel_empty = is_travel & (w3['load_state'] == 'EMPTY')
travel_priority_eligible = is_travel & (w3['priority_num'] >= int(REQUEST_PRIORITY))
asd_eligible = is_asd & (w3['as_directed_duration_min'] >= MIN_AS_DIRECTED_MIN)

w4 = w3[travel_empty | travel_priority_eligible | asd_eligible].copy()
c4 = len(w4)

# Pickup geography
if 'distance_to_pickup_mi' not in w4.columns:
    w4['distance_to_pickup_mi'] = w4.apply(
        lambda r: haversine_miles(r.get('from_lat'), r.get('from_lon'), origin_lat, origin_lon), axis=1
    )

w5 = w4[
    w4['distance_to_pickup_mi'].isna()
    | (w4['distance_to_pickup_mi'] <= PICKUP_RADIUS_MI_V2)
].copy()
c5 = len(w5)

# Home base from first START FACILITY on effective day
starts_v2 = work[(work['element_type_norm'].str.contains('START FACILITY', na=False)) & (work['active_effective'])].copy()
starts_v2 = starts_v2.sort_values([col_duty, 'start_min']).drop_duplicates(subset=[col_duty], keep='first')
home_base_v2 = starts_v2[[col_duty, 'from_norm', 'from_lat', 'from_lon']].rename(columns={
    'from_norm': 'home_base_name',
    'from_lat': 'home_base_lat',
    'from_lon': 'home_base_lon',
})

w6 = w5.merge(home_base_v2, on=col_duty, how='left')
w6['home_base_to_delivery_mi'] = w6.apply(
    lambda r: haversine_miles(r.get('home_base_lat'), r.get('home_base_lon'), delivery_lat, delivery_lon), axis=1
)

# STRICT: exclude unknown home-base-to-delivery distances from consideration
w6_known_home = w6[w6['home_base_to_delivery_mi'].notna()].copy()
c6_known = len(w6_known_home)

final_first_pass_v2 = w6_known_home[
    w6_known_home['home_base_to_delivery_mi'] <= HOME_BASE_TO_DELIVERY_RADIUS_MI_V2
].copy()
c6 = len(final_first_pass_v2)

# Display waterfall + sample
waterfall_v2 = pd.DataFrame([
    {'stage': 'all_rows', 'rows': c0},
    {'stage': 'active_effective', 'rows': c1},
    {'stage': 'travel_or_as_directed', 'rows': c2},
    {'stage': f'time_window_±{TIME_WINDOW_MIN}m', 'rows': c3},
    {'stage': 'priority/empty/as_directed>=30 eligible', 'rows': c4},
    {'stage': f'pickup_radius<= {PICKUP_RADIUS_MI_V2}mi', 'rows': c5},
    {'stage': 'known_home_base_to_delivery', 'rows': c6_known},
    {'stage': f'home_base_to_delivery<= {HOME_BASE_TO_DELIVERY_RADIUS_MI_V2}mi', 'rows': c6},
])

print(f"Scenario: {REQUEST_FROM} -> {REQUEST_TO} @ {REQUEST_WHEN_LOCAL}, request_priority={REQUEST_PRIORITY}")
display(waterfall_v2)

view_cols_v2 = [
    col_duty,
    col_operator,
    col_vehicle,
    'element_type_norm',
    'priority_num',
    'load_state',
    'as_directed_duration_min',
    'distance_to_pickup_mi',
    'home_base_name',
    'home_base_to_delivery_mi',
    'time_gap_min',
]
view_cols_v2 = [c for c in view_cols_v2 if c in final_first_pass_v2.columns]

final_first_pass_v2.sort_values(
    ['distance_to_pickup_mi', 'time_gap_min', 'home_base_to_delivery_mi'],
    na_position='last'
)[view_cols_v2].head(100)

Scenario: BIRMINGHAM MAIL CENTRE -> MIDLANDS SUPER HUB @ 2025-09-02T10:30, request_priority=3


,stage,rows
0,all_rows,73900
1,active_effective,35604
2,travel_or_as_directed,12472
3,time_window_±120m,1580
4,priority/empty/as_directed>=30 eligible,1508
5,pickup_radius<= 10mi,215
6,known_home_base_to_delivery,205
7,home_base_to_delivery<= 30mi,5


,Duty ID,Operator,Vehicle Type,element_type_norm,priority_num,load_state,as_directed_duration_min,distance_to_pickup_mi,home_base_name,home_base_to_delivery_mi,time_gap_min
195,SWDC014,SWDC,95 Artic DD,TRAVEL,3,LOADED,0,0.0,SOUTH WEST HUB,24.106962,3
196,SWDC209,SWDC,95 Artic DD,TRAVEL,3,LOADED,0,0.0,SOUTH WEST HUB,24.106962,93
0,BRVOC004,BRVOC,95 Artic DD,TRAVEL,3,LOADED,0,0.0,BRIDGEND VOC,14.626448,105
1,BRVOC204,BRVOC,49 Artic,TRAVEL,3,LOADED,0,NaN,BRIDGEND VOC,14.626448,30
197,SWDC229,SWDC,24 17t Rigid,TRAVEL,3,LOADED,0,NaN,SOUTH WEST HUB,24.106962,30
